In [ ]:
def load_samples(self, file_name):
        '''
        input: filename of samples
        output: distionary, key=patient name, value=0 healthy, 1 diseased
        '''
        d = {}
        with open(file_name, "r") as fh:
            lines = fh.readlines()
            for line in lines:
                tokens = line.split()
                d[tokens[0]] = tokens[1]
        #print("d:",d)
        return d

rows,cols = df.shape
#build healthy and diseased indexes
print(df.columns)
healthy_idx=[]
disease_idx=[]
for idx,col in enumerate(df.columns):
    if col!="SYMBOL":
        print("idx:",idx,"col:",col,samp[col])
        if samp[col]=='1':
            disease_idx.append(idx)
        elif samp[col]=='0':
            healthy_idx.append(idx)
        else:
            print("xxxx")
    
print("healthy_idx:",healthy_idx," disease_idx:",disease_idx)
df_np = df.to_numpy()
print("rows:",rows,"cols:",cols)#fetch healthy as list of columns, diseased as list of columns
rank_me=[]
healthy_mean=np.mean(df_np[0,[healthy_idx]])
disease_mean=np.mean(df_np[0,[disease_idx]])
for idx in range(0,df_np.shape[0]):
    #print("df iloc:",df.iloc[idx,0],df_np[0,[healthy_idx]],healthy_mean)
    #print(df_np[0,[disease_idx]],disease_mean)
    #print("diff:",disease_mean-healthy_mean)
    healthy_mean=np.mean(df_np[idx,[healthy_idx]])
    disease_mean=np.mean(df_np[idx,[disease_idx]])
    rank_me.append((idx,df.iloc[idx,0],disease_mean-healthy_mean)) #sort by second tuple value

print("length ranked genes: ",len(rank_me))
#before sorting
print("rank_me[0:5]",rank_me[0:5])
largest_first = sorted(rank_me, key = lambda x: x[2],reverse=True)
print("largest_first[0:5]",largest_first[0:5])
return_me=[]
for x in largest_first:
    return_me.append(x[1])
print("return_me[0:5]:",return_me[0:5])
print("len(largest_first): ",len(largest_first))
#print("PDHA1 in return_me:",("PDHA1" in return_me),len("PDHA1"))
#adjust gene remove

val_gene=list(pathway_gene_dict.values())
print("val_gene:",val_gene[0:5])
newlist = sum(val_gene, [])
print("newlist:",newlist[0:5])
#set_val_gene=set(newlist)

num=0
for x in range(0,len(newlist)):
    print("processing newlist:",newlist[x])
    if newlist[x] not in return_me:
        print("newlist[x] not in return_me:",newlist[x])
        num+=1
    else:
        print("newlist[x] in return_me:",newlist[x])

print("num not in return_me:",num)
#print("num val_gene:",len(val_gene),len(newlist))

print("KEGG_CITRATE_CYCLE_TCA_CYCLE",len(pathway_gene_dict['KEGG_CITRATE_CYCLE_TCA_CYCLE']))
#print(np.sqrt((10702-24)/24))
nt=10702

enrich=[]
for x in pathway_gene_dict.keys():
    ng = len(pathway_gene_dict[x])
    print("pathway:",x," ng:",len(pathway_gene_dict[x]))
    
    penalty_add =np.sqrt((nt-ng)/ng)
    penalty_miss = np.sqrt(ng/(nt-ng))
    #print("ng:",ng," penalty_add:",penalty_add," penalty_miss:",penalty_miss,len(return_me))    
    #calculate ES for each geneset pathway_gene_dict[x] using ranked list. wow for all 10k? 
    es=[]
    es.append(0.)
    sum_es=0.
    for idx in range(0,len(return_me)):
        #print("processing gene:",return_me[idx])
        #this is right list? not largest first???
        if return_me[idx] in expr_gene:
            if return_me[idx] in pathway_gene_dict[x]:
                sum_es +=penalty_add
            else:
                sum_es-=penalty_miss
            es.append(sum_es)
    print("es:",es)
    print("max:",max(es))
    enrich.append((x,max(es)))
print("enrichment scores:",enrich[0:5])
for x in enrich:
    if x[0]=="KEGG_CITRATE_CYCLE_TCA_CYCLE":
        print("KEGG_CITRATE_CYCLE_TCA_CYCLE",x)
largest_enrichment=sorted(enrich, key = lambda x: x[1],reverse=True)
print("largest enrichment scores:",enrich[0:5])

